# Milestone 3



## Introduction

Differentiation is ubiquitous in almost all aspects of computer science, mathematics, and physics. It is used for numeric root-finding as in Newton's Method, and used for optimization with different forms of gradient descent.
However, calculating analytic derivatives is difficult and can lead to exponentially growing abstract syntax trees, which makes finding the derivative infeasible in many cases. Calculating the derivative numerically using the limit definition runs into numeric instabilities due to limited machine precision. Automatic differentiation addresses both of these issues - it uses the chain rule and the fact that computers calculate any function as a sequence of elementary operations to find the derivative.

The package can also evaluate the Hessian of multivariable functions and higher order derivatives of scalar functions. This is particularly useful for optimization problems, where convergence on a function minimum is often significantly faster when Hessian information is included in the root finding algorithm. This is useful for Control Theory applications, as well as many important algorithms used in robotics. Calculating the Hessian is useful for path planning, as well as for inverse kinematics. Since it is relatively difficult to get a closed form solution for the Hessian in complicated inverse kinematics problems, it is useful to be able to use Automatic Differentiation for the Hessian.

## Installation and Usage


### Installing from Source

If you want the latest nightly version of AutoDiffX, clone from our github
repository and install the latest version directly.

```bash
git clone https://github.com/CS207-Project-Team-1/cs207-FinalProject autodiffx
cd autodiffx
pip install -r requirements.txt
python3 setup.py install
```

If you are working on a python virtual environment or Mac OSX or your user's
python distribution, this should work. If editing the system python, you may
need to run the last command with root permissions by adding `sudo`.

### Installing from pip

For the stable version, you can install our package from PyPI.

```bash
pip install autodiffx
```

## Testing

All of the tests are run using pytest. To run pytest, you want to be in the
root directory of the repository. To ensure that `pytest` gets the imports
correct, you want to run it such that it adds the current path to `PYTHONPATH`.
The easiest way to do so is:

```bash
python -m pytest
```

This should run all of the tests for the package.

Currently, our only module that we actually use lives in the `ad/` folder. The tests can be found in the different test files in `ad/tests`.




## Background

**Automatic Differentiation**

Automatic differentiation relies heavily on the principles of chain rule differentiation. A graph of elementary functions is built to calculate the values of more complex functions. Using the chain rule on the graph of elementary functions, the value of the derivative at each node can also be calculated. This gives us the ability to calculate the values of functions and their derivatives, no matter how complex, to near machine precision (a significant advantage compared to alternatives such as finite differences). 

The chain rule tells us that:
\begin{align}
\frac{d}{dx}(f(g(x))) &= f'(g(x))g'(x)\\
\end{align}

Since each step in our graph is just a combination of linear operations, we can find the derivative at a node by considering the value and derivative of the expressions at the previous node. By starting with an initial 'seed' vector for the derivative (often set to 1), we can find the derivative in any desired search direction. 

Below is an example of constructing a graph to find the exact values of a function and its derivative. The function we used was:

$$f\left(x, y, z\right) = \dfrac{1}{xyz} + \sin\left(\dfrac{1}{x} + \dfrac{1}{y} + \dfrac{1}{z}\right)$$

We worked through this, starting with trace elements $x_1$ for $x$,  $x_2$ for $y$ and  $x_3$ for $z$. We wanted to solve this function at $(x, y, z) = (1, 2, 3)$.

| Trace | Elementary Function | Current Value | Elementary Function Derivative | $\nabla_{x}$ Value  | $\nabla_{y}$ Value  | $\nabla_{z}$ Value  |
| :---: | :-----------------: | :-----------: | :----------------------------: | :-----------------: | :-----------------: | :-----------------: |
| $x_{1}$ | $x$ | 1 | $\dot{x}$ | 1 | 0 | 0 | 
| $x_{2}$ | $y$ | 2 | $\dot{y}$ | 0 | 1 | 0 | 
| $x_{3}$ | $z$ | 3 | $\dot{z}$ | 0 | 0 | 1 | 
| $x_{4}$ | $1/x_{1}$ | 1 | $-\dot{x}_{1}/x_{1}^{2}$ | $-1$ | $0$ | $0$ | 
| $x_{5}$ | $1/x_{2}$ | 1/2 | $-\dot{x}_{2}/x_{2}^{2}$ | $0$ | $-1/4$ | $0$ | 
| $x_{6}$ | $1/x_{3}$ | 1/3 |  $-\dot{x}_{3}/x_{3}^{2}$ | $0$ | $0$ | $-1/9$ | 
| $x_{7}$ | $x_{4} + x_{5} + x_{6}$ | 11/6 |$\dot{x}_{4} + \dot{x}_{5} + \dot{x}_{6}$ | -1 | -0.25 | -0.11 |
| $x_{8}$ | $sin(x_{7})$ | 0.966 |$\dot{x}_{7}cos(x_{7})$ | 0.260 | 0.065 | 0.029 | 
| $x_{9}$ | $x_{4}x_{5}x_{6}$| 1/6 |$\dot{x}_{4}x_{5}x_{6} + \dot{x}_{5}x_{4}x_{6} + \dot{x}_{6}x_{4}x_{5} $ |-0.167 | -0.083  | -0.056 | 
| $x_{10}$ | $x_{8} + x_{9}$ | 1.132 |$\dot{x}_{8} + \dot{x}_{9}$ | 0.093| -0.018  | -0.027 | 

This isn't a very complicated function, but it shows how we can use the most basic of functions to create a graph allowing us to find exact values and gradients.

**Higher-Order Derivatives**

An effective approach to high-order automatic differention can be obtained by considering the calculus of a Taylor series. If we define $f_k$ as follows:

$$f_k = f_{k}(x_0) = \dfrac{f^{(k)}(x_0)}{k!}$$

We can show that basic arithmetic operations are as follow, where $f$ and $g$ are separate functions with the same input variable. Full derivations of these basic operations can be found [here](https://www.sintef.no/globalassets/project/evitameeting/2010/ad2010.pdf):

$$(f + g)_k = f_k + g_k$$

$$(f - g)_k = f_k - g_k$$

$$(f \times g)_k = \sum_{i = 0}^{k} f_{i}g_{k-i} $$

$$(f \div g)_k = \dfrac{1}{g_0} \left(f_k - \sum_{i = 0}^{k - 1} (f \div g)_{i} g_{k-i}\right) $$

$$ (e^g)_k = \dfrac{1}{k} \sum_{i = 1}^{k} ig_{i}(e^g)_{k-i}$$

**Hessian**

## Simple Usage

Our package supports scalar functions with scalar or multivariable inputs. 

**Scalar Input**

Suppose that we wanted to find the derivative of the following function, $f(x)$:

$$f(x) = x \exp(\cos(x) + x^2)$$

First, we have to allocate a Variable for our input variable $x$. We can do that, and give it a string identifier `"x"` for convenience later. 

```python
import ad
x = ad.Variable('x')
```
Now, we have to define our function. Our module `ad` has a lot of built in functions, and all of the regular operators `+`, `-`, `*`, `/`, `**` should also work. We can make our function $f(x)$ by just writing it out in code.

```python
f = x * ad.Exp(ad.Cos(x) + x ** 2)
```

Now, we've defined our function in terms of our input variable $x$. Now, in order to evaluate our function or evaluate the derivative at a specific point we actually have to provide a value for the input. Since we later plan on handling multiple variable inputs, we pass the input in as a dictionary. To evaluate the function, we use the function ```eval```. For the derivative at a point, we use ```d```. Suppose that we wanted to evaluate the function $f$ and its derivative at $x = 0$ and $x = 1$. We can just run:

```python
>>> f.eval({x: 0})
2.718281828459045

>>> f.d({x: 0})
1.0

>>> f.eval({x: 1})
5.666000617166735

>>> f.d({x: 1})
6.405697099891925
```

It is also possible to evaluate higher order derivatives of functions with scalar inputs. This can be done using ```f.d_n(n, val)``` where n and val are arguments expressing the desired order of differentiation and the value at which the derivative should be evaluated. We can run:

```python
>>> f.d_n(n = 3, val = 2)
1902.7256925837773
```

**Multi-Variable Input**

Suppose that we wanted to find the derivative of the following function, $f(x, y)$:

$$f(x) = x \exp(\cos(y) + x^2)$$

We have to allocate a variables for our inputs $x$ and $y$. We can do that, and give them string identifiers `"x"` and `"y"` for convenience later. 

```python
import ad
x = ad.Variable('x')
y = ad.Variable('y')
```
We now define our function.

```python
f = x * ad.Exp(ad.Cos(y) + x ** 2)
```

We can evaluate our function and it's derivative exactly as we evaluated our scalar input example earlier. Suppose that we wanted to evaluate the function $f$ and its derivative at $x = 0$ and $y = 1$. We can just run:

```python
>>> f.eval({x: 0, y : 1})
1.7165256995489035

>>> f.d({x: 0, y : 1})
{y: -1.4444065708474794, x: 1.0}
```

Using the Hessian is very similar to calling the derivative of a multivariable function. If the function depends on $n$ input variables, the package will return an $n \times n$ matrix in the form of a dictionary of dictionaries, where each sub-dictionary refers to one of the rows in the matrix. 

Hence, for our multivariable example above, $f(x, y)$:

$$f(x) = x \exp(\cos(y) + x^2)$$

We can evaluate the Hessian by calling:

```python
>>> f.hessian({x: 0, y : 1})
{y: {y: 0.28798342608583105, x: 0.0}, x: {y: 0.0, x: 3.433051399097807}}
```
If we want the specific second order derivatives, we can evaluate specific elements from the Hessian by calling the keys. For example, if we wanted to find the second order derivative of our function with respect to $x$, we would call:

```python
>>> f.hessian({x: 0, y : 1})[x][x]
 3.433051399097807
```

More complicated demonstrations (scalar input Newton's Method, multivariable input Newton's Method with Hessian) can be found in Jupyter notebooks at the top level directory. 

## Software Organization

## Directory structure

```bash
cs207project
├── LICENSE
├── README
├── docs
│   ├── Milestone1.ipynb
│   ├── Milestone2.ipynb
│   ├── Milestone3.ipynb 
│   └── SETUP.md
├── ad
│   ├── __init__.py
│   ├── ad.py
│   ├── simple_ops.py
│   ├── activation_ops.py.../
│   └── tests
│       ├── test_complex_ops.py
│       ├── test_d_expr.py
│       ├── test_expression.py
│       ├── test_high_order.py
│       ├── test_multivar_hessian.py
│       ├── test_simple_hessian.py
│       ├── test_simple_ops.py
│       └── test_vector.py
├── requirements.txt
├── Newton_Method_Demonstration.ipynb
├── Hessian_Demonstration.ipynb
└── setup.py
```

## Modules

`ad`: Implementation of core structures used in our graph-based automatic differentiation library. +, -, *, /, and exponentiation is also implemented here to support simple operator overloading.

`simple_ops`: Unary operations including Sin(x), Cos(x), Tan(x), Sinh(x), Cosh(x), Tanh(x), Exp(x), Log(x), Arcsin(x), Arccos(x), Arctan(x), Logistic(x).

## Test

See Testing.

## Installment

See Installation and Usage



## Implementation Details

We implemented our automatic differentiation using a graph based structure. First, the user will build up their function in terms of elementary operations. Then, the user will be able to feed a value for their input variables, and the package will calculate the derivatives at each step.

### Automatic Differentiation Basics

Our core data structure is the computational graph. Every node in the computational graph would be an "Expression", which is our core class. 

Based on "Expression", we have "Variable(Expression)", "Constant(Expression)", "Unop(Expression)" and "Biop(Expression)". "Unop" is for unary operations such as log and power. "Biop" is for binary operations such as addition. For "Expression" and its subclasses, there are two important attributes: "grad", a boolean variable indicating whether we want to calculate the derivative or not; children, a list of nodes pointing to current one, the number of children is one for "Unop" and the number of chilren for "Biop" is two.

The elementary functions we support are:

Unary operations: 

* Sin(x)
* Cos(x)
* Tan(x)
* Sinh(x)
* Cosh(x)
* Tanh(x)
* Exp(x)
* Log(x)
* Arcsin(x)
* Arccos(x)
* Arctan(x)
* Logistic(x)

Binary operations: 

* Addition (+)
* Substraction (-)
* Multiplication (*)
* Division (/)
* Power(x, n)

We will mainly be using `numpy` as an external dependency for mathematical and vector operations.




### Multivariate Inputs

We want to also support getting the Jacobian when we have more than one input for a scalar function. We are able to support functions of the form
$$f: \mathbb{R}^m \to \mathbb{R}$$
Our implementation for multivariate inputs is by using a `dict` to hold the different partial derivatives. We can go through an example. Suppose that a user wanted the Jacobian of a function:

$$f(x, y, z) = x  + y \cos(yz)$$

Then, we would be able to find the Jacobian in code using the following syntax.

```python
>>> x = ad.Variable('x')
>>> y = ad.Variable('y')
>>> z = ad.Variable('z')

>>> f = x + y * ad.Cos(y * z)

>>> f.eval({x:6, y:1, z:0})
7
>>> f.d({x:6, y:1, z:0})
{x: 1, y: 1, z: 0}
```


### Extension Feature - Higher Order Derivatives and Hessian

**Higher Order Derivatives**

See background.

**Hessian**

Our package implements the Hessian for functions of the form
$$f: \mathbb{R}^m \to \mathbb{R}$$
In this case, the Hessian should be a square matrix. Since the Hessian will be a square matrix with entries of the form
$$\frac{\partial^2 f}{\partial x_i \partial x_j}$$
for some $i, j$, we return a dictionary of dictionaries for the Hessian. This makes it easy to index the Hessian as ```hessian[x][y]``` to get the second order derivative with respect to x and y. An example function and calculated Hessian will be look something similar to the follows.

$$f: \mathbb{R}^3 \to \mathbb{R}$$
$$f(x, y, z) = xy + z$$

Then, the code uses the function `hessian()` to calculate the Hessian at a certain point.

```python
>>> x = ad.Variable('x')
>>> y = ad.Variable('y')
>>> z = ad.Variable('z')

>>> f = x * y + z

>>> f.eval({x:1, y:1, z:1})
2

>>> f.hessian({x:1, y:1, z:1})
{x: {x: 0, y: 1, z:} y: {x: 1, y: 0, z: 0} z: {x: 0, y: 0, z: 0}}
```

### Future Extensions

**Multivariate Outputs**

Currently, our package only fully supports scalar functions with scalar or multivariate input.

However, the next step after supporting multivariate inputs is to also support multivariate outputs. We will create a wrapper class `ad.Vector` that essentially wraps multiple different `ad.Expression` instances together. This will allow us to combine multiple scalar functions into a multivariate function. For example, suppose that a user wanted to find the Jacobian of a function in the form:
$$f: \mathbb{R}^3 \to \mathbb{R}^3$$
and suppose that the function was in the form:
$$f(x, y, z) = 
\begin{pmatrix}
x + y + z \\
\cos(yz) \\
\exp(x - y - z)
\end{pmatrix}
$$

This could be represented in code as follows:

```python
>>> x = ad.Variable('x')
>>> y = ad.Variable('y')
>>> z = ad.Variable('z')

>>> f1 = x + y + z
>>> f2 = ad.Cos(y * z)
>>> f3 = ad.Exp(x - y - z)

>>> f = ad.Vector(f1, f2, f3)
>>> f.eval({x:0, y:0, z:0})
[0, 1, 1]

>>> f.d({x:0, y:0, z:0})
{x: [1, 0, 1], y: [1, 0, -1], z: [1, 0, -1]}
```